In [1]:
import pandas as pd
import plotly.express as px
from pathlib import Path

## Step 9: Loading Cleaned Data

In [ ]:
df = pd.read_csv('ucs_satellites_derived.csv', parse_dates = ['date_of_launch'])

In [ ]:
shell_counts = df['orbit_shell'].value_counts().rename_axis('orbit_shell').reset_index(name = 'count')
print("\nCounts by shell:\n", shell_counts.to_string(index=False))


Counts by shell:
 orbit_shell  count
        LEO   6768
        GEO    590
        MEO    143
    UNKNOWN     59


## Step 10: Visualising Data

In [ ]:
# Chart 1: Total Satellites by Orbit Shell
shell_counts = df.groupby('orbit_shell').size().reset_index(name = 'count')

fig1 = px.bar(
    shell_counts,
    x = 'orbit_shell',
    y = 'count',
    title = 'Total Satellites by Orbit Shell',
    color = 'orbit_shell', 
    color_discrete_sequence = px.colors.sequential.Plasma
)
fig1.update_layout(
    xaxis_title = 'Orbit Shell',
    yaxis_title = 'Satellite Count',
    showlegend = False,
    template = 'plotly_white'
)
fig1.show()


In [92]:
# Chart 2: Launches per Year by Orbit Shell
ts = df.dropna(subset = ['launch_year']).groupby(['launch_year','orbit_shell']).size().reset_index(name = 'count')

fig2 = px.line(
    ts,
    x = 'launch_year',
    y = 'count',
    color = 'orbit_shell',
    title = 'Satellite Launches per Year by Orbit Shell',
    markers = True,
    color_discrete_sequence = px.colors.qualitative.Set1
)
fig2.update_layout(
    xaxis_title = 'Year',
    yaxis_title = 'Launches',
    template = 'plotly_white'
)
fig2.show()


In [87]:
# Chart 3: Top 10 Operators by Satellite Count
top_ops = df['operator'].value_counts().nlargest(10).reset_index()
top_ops.columns = ['operator', 'count']


fig3 = px.bar(
    top_ops, 
    x = 'operator',
    y = 'count', 
    title = 'Top 10 Operators by Number of Satellites',
    color = 'operator',   
    color_discrete_sequence = px.colors.sequential.Bluered,
    text = 'count'
)
fig3.update_layout(xaxis_tickangle = -45, 
    showlegend = False,
    xaxis_title = 'Satellite Count',
    yaxis_title = 'Operator',
    template = 'plotly_white',
)

fig3.show()


In [82]:
# Chart 4: Satellite Age Distribution by Orbit Shell
import datetime
current_year = datetime.datetime.now().year
df['sat_age'] = current_year - df['launch_year']

fig4 = px.box(
    df.dropna(subset=['sat_age']),
    x="orbit_shell",
    y="sat_age",
    color="orbit_shell",
    title="Satellite Age Distribution by Orbit Shell",
    color_discrete_sequence=px.colors.qualitative.Pastel2
)
fig4.update_layout(
    yaxis_title="Satellite Age (Years)",
    xaxis_title="Orbit Shell",
    showlegend=False,
    template="plotly_white"
)
fig4.show()

In [75]:
# Chart 5: Average Satellite Age by Operator
op_age['operator_short'] = op_age['operator'].apply(lambda x: x if len(x) <= 25 else x[:25] + "...")

fig5 = px.bar(
    op_age,
    x = 'mean',
    y = 'operator_short',  
    orientation = 'h',
    text = 'mean',
    color = 'mean',
    color_continuous_scale = 'Blackbody',
    title = 'Average Satellite Age by Operator (Top Operators)',
    hover_data = {'operator': True, 'mean': True}  
)
fig5.update_traces(texttemplate = '%{text:.1f} yrs', textposition = 'outside')

fig5.update_layout(
    xaxis_title = 'Average Age (Years)',
    yaxis_title = 'Operator',
    template = 'plotly_white',
    coloraxis_showscale = False,
    margin = dict(l = 200)  
)
fig5.show()

In [78]:
# Chart 6: Incidence of Old Hardware
old_hw = df.groupby('orbit_shell').apply(
    lambda x: (x['sat_age'] > 15).mean()*100
).reset_index(name = 'pct_old')

fig6 = px.bar(
    old_hw,
    x = 'orbit_shell',
    y = 'pct_old',
    text = 'pct_old',
    color = 'pct_old',
    color_continuous_scale = 'Rainbow',
    title = 'Incidence of Old Hardware by Orbit Shell'
)

fig6.update_traces(texttemplate = '%{text:.1f}%', textposition = 'outside')
fig6.update_layout(
    xaxis_title = 'Orbit Shell',
    yaxis_title = 'Percent of Old Satellites',
    template = 'plotly_white',
    coloraxis_showscale = False,
    font=dict(size=10)
)
fig6.show()

/var/folders/nt/4rp641zs5lg7_234f6d1h4_00000gn/T/ipykernel_16756/3146790279.py:2: FutureWarning:

DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.



In [81]:
# Chart 7: Dominant Orbit Shell by Country
# Getting dominant orbit per country
dom_shell = df.groupby(['country_operator','orbit_shell']).size().reset_index(name = 'count')
dom_shell = dom_shell.loc[dom_shell.groupby('country_operator')['count'].idxmax()]

fig7 = px.choropleth(
    dom_shell,
    locations = 'country_operator',
    locationmode = 'country names',
    color = 'orbit_shell',
    title = 'Dominant Orbit Shell by Country',
    color_discrete_sequence = px.colors.qualitative.Pastel1
)
fig7.update_layout(template = 'plotly_white')
fig7.show()


/var/folders/nt/4rp641zs5lg7_234f6d1h4_00000gn/T/ipykernel_16756/2983319651.py:6: DeprecationWarning:

The library used by the *country names* `locationmode` option is changing in an upcoming version. Country names in existing plots may not work in the new version. To ensure consistent behavior, consider setting `locationmode` to *ISO-3*.

